# Implementando classe para fazer o GSC separadamente

Como uma classe para fazer apenas o DaS foi criada, o GSC também vai precisar de uma classe nova. A ideia é usar os sinais dos microfones e esse sinal que vai vir dessa classe do DaS. 

In [1]:
from IPython.display import Audio
from scipy.optimize import least_squares
from random import random
import numpy as np
import sys
sys.path.append("/home/dimi/Programming/IC2019/DOA/Python/")
from DaSParallelClass import DaSParallel
from SimularArrayMicrofonesClass import SimularArrayMicrofones

### Abrindo um áudio para começar os testes

Já vou simular um array de microfones com ele e aplicar o DaS.

In [2]:
# COORDENADAS 
qtdMics = 10
coordenadasMics = []
for i in range(qtdMics):
    coordenadasMics.append([random()/10, random()/10, random()/10])
coordenadasMics = np.array(coordenadasMics)
for i, coordenadaAtual in enumerate(coordenadasMics):
    coordenadasMics[i] = coordenadaAtual - coordenadasMics[0]

# SIMULO UM ARRAY DE MICS
caminho = "/home/dimi/Programming/IC2019/DOA/Gravacoes/Simulacoes/Gravacao4/lars_maynard.wav"
objSimularArrayMics = SimularArrayMicrofones(caminho, energiaRuido=0.25, normalizarEnergia=True, coordenadasMics=coordenadasMics)
arraySinaisMics, arrayDelaysVerdadeiros, freqAmostragem = objSimularArrayMics.obterResultado()
    
# APLICO O DAS
objDaS = DaSParallel(arraySinaisMics, qtdNucleos=1)

In [3]:
print("SEM DaS")
Audio(data=np.sum(arraySinaisMics, axis=0)/len(arraySinaisMics), rate=freqAmostragem)

SEM DaS


In [4]:
print("COM DaS")
Audio(data=objDaS.sinalDaS, rate=freqAmostragem)

COM DaS


## Implementando o GSC

Para iniciar a implementação eu só vou precisar dos sinais dos microfones (arraySinaisMics) e do sinal após o DaS (sinalDaS).

### Função para cortar um pedacinho dos sinais dos microfones

O DaS implementado come um pouquinho do sinal captado pelos microfones (bem pouquinho). Pras operações que eu vou fazer em seguida darem certo, preciso deixar o tamanho dos sinais igual.

In [5]:
def cortarSinais(arraySinaisMics, tamanho):
    return arraySinaisMics[:,0:tamanho]

In [6]:
#arraySinaisMics = np.array(arraySinaisMics)
print("Tamanho do sinal após o DaS:", len(objDaS.sinalDaS))
print("Shape dos sinais dos mics antes da função:", arraySinaisMics.shape)
arraySinaisMics = cortarSinais(arraySinaisMics, len(objDaS.sinalDaS))
print("Shape dos sinais dos mics após a função:", arraySinaisMics.shape)

Tamanho do sinal após o DaS: 210894
Shape dos sinais dos mics antes da função: (10, 210944)
Shape dos sinais dos mics após a função: (10, 210894)


### Função pra gerar a blocking matrix

In [7]:
def gerarBlockingMatrix(qtdMics):

    blockingMatrix = np.zeros((qtdMics-1, qtdMics))

    for cont, linha in enumerate(blockingMatrix):
        linha[cont]   = 1
        linha[cont+1] = -1

    return blockingMatrix

In [8]:
blockingMatrix = gerarBlockingMatrix(len(arraySinaisMics))
print(blockingMatrix)

[[ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -1.]]


### Função para passar os sinais dos mics pela blocking matrix

In [9]:
def obterSinaisRuidosos(blockingMatrix, arraySinaisMics):
    return np.matmul(blockingMatrix, arraySinaisMics)

In [10]:
arraySinaisRuidosos = obterSinaisRuidosos(blockingMatrix, arraySinaisMics)
print(arraySinaisRuidosos.shape)

(9, 210894)


In [11]:
print("Exemplo de sinal que sobra depois da blocking matrix:")
Audio(data=arraySinaisRuidosos[0], rate=freqAmostragem)

Exemplo de sinal que sobra depois da blocking matrix:


### Função que será minimizada pelo LMS

In [12]:
# SETANDO PARAMETROS QUE DEVERAO SER GLOBAIS A CLASSE
sinalDaS            = objDaS.sinalDaS
arraySinaisRuidosos = arraySinaisRuidosos

In [13]:
def calcularEnergiaSinalFinal(arrayPesos):    
    sinalFinalRuidoso = np.zeros(len(sinalDaS))
    for index, pesoAtual in enumerate(arrayPesos):
        sinalFinalRuidoso += pesoAtual * arraySinaisRuidosos[index]
    sinalFinal = sinalDaS - sinalFinalRuidoso
    return np.var(sinalFinal)

In [14]:
arrayPesos = np.ones(len(arraySinaisRuidosos))
energiaSinalFinal = calcularEnergiaSinalFinal(arrayPesos)
print(np.var(objDaS.sinalDaS))
print(energiaSinalFinal)

1.0256153070626985
2.0285928687569488


### Função que vai iterar o LMS para encontrar o array de pesos

In [15]:
def obterArrayPesos(qtdSinaisRuidosos):
    chuteInicial      = np.full(qtdSinaisRuidosos, 0).tolist()
    limitesInferiores = np.full(qtdSinaisRuidosos, -100).tolist()
    limitesSuperiores = np.full(qtdSinaisRuidosos, 100).tolist()

    # RODANDO O LMS. ELE RETORNA UM OBJETO CHEIO DE ATRIBUTOS
    objRespostaLMS = least_squares(fun=calcularEnergiaSinalFinal, x0=chuteInicial, bounds=[limitesInferiores, limitesSuperiores], verbose=0)

    # O UNICO ATRIBUTO QUE ME INTERESSA E ESSE "X", QUE E A RESPOSTA ENCONTRADA DE FATO
    return objRespostaLMS.x

In [16]:
arrayPesos = obterArrayPesos(len(arraySinaisRuidosos))
print(arrayPesos)

[-0.07238254 -0.17619098 -0.2852017  -0.21712759 -0.21644496 -0.19751059
 -0.12476745  0.16510336 -0.08507341]


### Função para gerar o sinal do GSC depois de encontrar os pesos

In [17]:
def gerarSinalFinalGSC(sinalDaS, arrayPesos, arraySinaisRuidosos):
    sinalFinalRuidoso = np.zeros(len(sinalDaS))
    for index, pesoAtual in enumerate(arrayPesos):
        sinalFinalRuidoso += pesoAtual * arraySinaisRuidosos[index]
    sinalFinal = sinalDaS - sinalFinalRuidoso

    return sinalFinal

In [18]:
sinalGSC = gerarSinalFinalGSC(objDaS.sinalDaS, arrayPesos, arraySinaisRuidosos)
print(np.var(sinalDaS))
print(np.var(sinalGSC))

1.0256153070626985
0.886052685212746


In [19]:
print("Sinal resultante do GSC:")
Audio(data=sinalGSC, rate=freqAmostragem)

Sinal resultante do GSC:
